In [0]:
from bs4 import BeautifulSoup
import requests
import urllib.request
import pandas as pd
import numpy as np
from pandas import Series, DataFrame
from tqdm import tqdm, tqdm_notebook
import time
import os
from datetime import datetime
import random
from threading import Thread

# 네이퍼 플레이스 리뷰 크롤링 사용방법
### 1. 아래 셀의 path, imgDir, start, end, headers 를 수정하여 주세요.
    - path : naver_map_seoul_salon_output_data_2020-04-19_final.csv 파일 경로
    - imgDir : 이미지 파일을 저장할 디렉토리(끝에 / 를 필수로 넣어주세요)
          **경로의 하위 디렉토리 자동으로 생성되게 만들었습니다.**
    - start : 시작할 인덱스 번호
    - end : 끝낼 인덱스 번호(ex> 3 입력 시 3번 index까지 크롤링 됩니다.)
          **end번호가 start번호와 100개 이상 차이가 나면 데이터를 긁어올 수 없는 오류 발견
          **현재까지 이유를 알 수 없어서 수정할 수가 없습니다
          **time.sleep을 주고 리뷰 데이터가 없는 업소로 판단되는 업소를 따로 빼서 저장합니다.
    - hearders : Chrome/80.0.3987.163 부분을 자신의 크롬 버젼에 맞게 수정하세요.
### 2. 수정이 끝났으면 함수 선언한 셀까지 Run 해주시기 바랍니다.
### 3. 하단의 크롤링 하고 싶은 리뷰를 실행해주세요!
### 4. 크롤링이 끝나면 자동으로 노트북 파일이 있는 디렉토리에 CSV파일로 저장됩니다.
    - reviewDF.to_csv("./naver_place_booking_review.csv", index=False) 이 명령어를 다른 셀로 빼주시고 따로 저장하셔도 무관합니다.
### 5. 오류가 생긴다면 구글 드라이브에 오류 난 부분을 복사해서 적어주시길 바랍니다!  

In [0]:
#naver_map_seoul_salon_output_data_2020-04-19_final.csv 파일 경로
path = './naver_place_count_total_2020-05-01_final.csv'
#이미지 파일 저장할 디렉토리 설정. 반드시 끝에 /를 포함하셔야 합니다.
# imgDir = 'd:/marytalk/image/reviews/'
#크롤링할 업소 인덱스 번호 설정
#자신의 크롬 버젼에 따라서 Chrome/80.0.3987.163 부분을 수정해주세요
headers = {"User-Agent" : "Mozilla/5.0 (Windows NT 10.0; Win64; x64) \
            AppleWebKit/537.36 (KHTML, like Gecko) Chrome/80.0.3987.163 Safari/537.36", 
            "Accept":"text/html,application/xhtml+xml,application/xml;\
            q=0.9,imgwebp,*/*;q=0.8"}

In [0]:
#업소 CSV파일 불러오기
salonDF = pd.read_csv(path)
salonDF = salonDF[['store_id_only_num', 'store_name']]

In [0]:
#예약자 리뷰 함수
def booking_crawler(start, end):
    if not os.path.isdir(imgDir):
            os.makedirs(imgDir)
    baseurl = 'https://store.naver.com/hairshops/detail?id='
    
    for code_index in tqdm(range(start, end+1)):
        page_number=0
        time.sleep(random.uniform(0,4)+np.random.rand())
        while 1 :
            booking_url = baseurl + str(salonDF.store_id[code_index])+'&tab=bookingReview&tabPage='+str(page_number)
            html = requests.get(booking_url, headers=headers)
            naver_place = BeautifulSoup(html.text, 'html.parser')   
            
            if html is None :
                print(f"{code_index} {salonDF.store_name[code_index]} {page_number} 페이지 접속 에러")
                time.sleep(30)
                    
            elif naver_place.find('ul', class_="list_booking_review") is None :
                print(f"{code_index} {salonDF.store_name[code_index]} 접속 성공")   
                reject_salon = {"store_id":salonDF.store_id[code_index] ,"store_name":salonDF.store_name[code_index]}
                rejected_salon_list.append(reject_salon)
                print(str(code_index) + " "+salonDF.store_name[code_index]+" 리뷰 데이터 없음")
                break

            else : 
                print(f"{code_index} {salonDF.store_name[code_index]} 접속 성공")   
                try :
                    lastpage = naver_place.find('span', class_='total').get_text()
                except AttributeError :
                    lastpage = 1
                reviews = naver_place.find('ul', class_="list_booking_review").find_all('li', class_=["list_item","list_item no_img"])
                for rev in tqdm(reviews) :
                    #하나의 리뷰에서 별점 가져오기 
                    score = rev.find('span', class_='score').get_text()

                    #하나의 리뷰에서 리뷰 가져오기
                    review = rev.find('div', class_='inner').get_text()

                    #하나의 리뷰에서 미용사 가져오기
                    try :
                        hairdresser = rev.find('span', class_='booking_name').get_text()
                    except AttributeError :
                        hairdresser = np.nan

                    #하나의 리뷰에서 서비스 가져오기
                    try :
                        service = rev.find('span', class_='procedure').get_text().lstrip(' - ')
                    except AttributeError :
                        service = np.nan

                    #하나의 리뷰에서 작성자 가져오기
                    reviewer_id = rev.find('span', class_='item name').get_text().rstrip('****')

                    #하나의 리뷰에서 방문일자 가져오기
                    visit_date = rev.find('span', class_='item date').get_text().rstrip(' 방문')
                    visit_date = datetime.strptime(visit_date,'%Y.%m.%d').strftime('%Y%m%d')

                    #pk
                    pk = salonDF.store_id[code_index]+'B'+str(visit_date)+reviewer_id+'_'+str(service)
                    pk = "".join(i for i in pk if i not in "\/:*?<>|")
                    rev_info = {'pk': pk, 'store_name': salonDF.store_name[code_index],'score':score, 'review':review, 
                                'hairdresser':hairdresser,'service':service, 'reviewer_id':reviewer_id, 'visit_date':visit_date}
                    review_list.append(rev_info)
                    if str(rev).split('"')[1] == 'list_item':
                        #하나의 리뷰에서 이미지 가져오기
                        imgurl = rev.find("img")["src"].split('&type=')[0]
                        urllib.request.urlretrieve(imgurl, imgDir + pk + ".png" )
                        print(str(code_index) + " "+salonDF.store_name[code_index]+" 이미지 저장 : " + pk)
                    elif str(rev).split('"')[1] == 'list_item ':
                        imgurl = rev.find("img")["src"].split('&type=')[0]
                        urllib.request.urlretrieve(imgurl, imgDir+ pk + ".png" )
                        print(str(code_index) + " "+salonDF.store_name[code_index]+" 왜 공백이지? 이미지 저장 : " + pk)
                    else :
                        print(str(code_index) + " "+salonDF.store_name[code_index] + " 이미지가 없는 리뷰 저장.")

                print(str(code_index) + " "+salonDF.store_name[code_index] + " " + str(page_number+1) + "/"+ str(lastpage) + " 페이지 완료")
                if str(page_number+1)==str(lastpage) :
                    print(str(code_index) + " "+salonDF.store_name[code_index] + " 리뷰 저장 완료")
                    break
                else :
                    page_number+=1   

In [0]:
#영수증 리뷰 함수            
def receipt_crawler(start, end):
    baseurl = 'https://store.naver.com/hairshops/detail?id='
    for code_index in tqdm_notebook(range(start, end)):
        page_number=0
        retry_count=0
        time.sleep(random.uniform(0,4)+np.random.rand())
        while 1 :
            receipt_url = baseurl + str(salonDF.store_id_only_num[code_index])+'&tab=receiptReview&tabPage='+str(page_number)
            html = requests.get(receipt_url, headers=headers)                
            naver_place = BeautifulSoup(html.text, 'html.parser')
            if html.status_code != 200 :
                retry_count+=1
                
            elif naver_place.find('ul', class_="list_receipt_review") is None :
                reject_salon = {"store_id_only_num":salonDF.store_id_only_num[code_index] ,"store_name":salonDF.store_name[code_index]}
                rejected_salon_list.append(reject_salon)
                print(str(code_index) + " "+salonDF.store_name[code_index]+" 리뷰 데이터 없음")
                break
            
            elif retry_count>10 :
                reject_salon = {"store_id_only_num":salonDF.store_id_only_num[code_index] ,"store_name":salonDF.store_name[code_index]}
                rejected_salon_list.append(reject_salon)
                print(str(code_index) + " "+salonDF.store_name[code_index]+" 리뷰 데이터 불러오기 실패")
                break
                
            else : 
                try :
                    lastpage = naver_place.find('span', class_='total').get_text()
                except AttributeError :
                    lastpage = 1
                reviews = naver_place.find('ul', class_="list_receipt_review").find_all('li', class_=["list_item","list_item no_img"])
                for rev in tqdm(reviews) :
                    #하나의 리뷰에서 별점 가져오기 
                    score = rev.find('span', class_='score').get_text()

                    #하나의 리뷰에서 리뷰 가져오기
                    review = rev.find('div', class_='review_txt').get_text()

                    #하나의 리뷰에서 작성자 가져오기
                    reviewer_nickname = rev.find('a', class_='item').get_text()

                    #하나의 리뷰에서 작성일자 가져오기
                    write_date = rev.find_all('span', class_='item')[0].get_text()
                    write_date = datetime.strptime(write_date,'%Y.%m.%d').strftime('%Y%m%d')
                    
                    #하나의 리뷰에서 방문 빈도수 가져오기
                    frequency = rev.find_all('span', class_='item')[1].get_text().rstrip('번째 방문')

                    #pk
                    #pk = salonDF.store_id_only_num[code_index]+'R'+str(write_date)+reviewer_nickname+str(frequency)
                    #pk = "".join(i for i in pk if i not in "\/:*?<>|")
                    rev_info = {'store_name': salonDF.store_name[code_index],'score':score, 'review':review, 
                                'reviewer_nickname':reviewer_nickname, 'write_date':write_date, 'frequency':frequency}
                    review_list.append(rev_info)
                    if str(rev).split('"')[1] == 'list_item':
                        print('이미지 있는 리뷰')
                        #하나의 리뷰에서 이미지 가져오기
                        #imgurl = rev.find("img")["src"].split('&type=')[0]
                        #urllib.request.urlretrieve(imgurl, imgDir + pk + ".png" )
                        #print(str(code_index) + " "+salonDF.store_name[code_index]+" 이미지 저장 : " + pk)
                    elif str(rev).split('"')[1] == 'list_item ':
                        print('이미지 있는 리뷰')
                        #imgurl = rev.find("img")["src"].split('&type=')[0]
                        #urllib.request.urlretrieve(imgurl, imgDir+ pk + ".png" )
                        #print(str(code_index) + " "+salonDF.store_name[code_index]+" 왜 공백이지? 이미지 저장 : " + pk)
                    else :
                        print(str(code_index) + " "+salonDF.store_name[code_index] + " 이미지가 없는 리뷰 저장.")

                print(str(code_index) + " "+salonDF.store_name[code_index] + " " + str(page_number+1) + "/"+ str(lastpage) + " 페이지 완료")
                if str(page_number+1)==str(lastpage) :
                    print(str(code_index) + " "+salonDF.store_name[code_index] + " 리뷰 저장 완료")
                    html.close()
                    break
                else :
                    page_number+=1 
    

In [0]:
def threading_craw(start, end, n_thread) :
    step = np.linspace(start, end, n_thread+1, dtype=int)
    ths = []
    
    for i in tqdm_notebook(range(0,n_thread)) :
        th = Thread(target=receipt_crawler, args=(step[i], step[i+1]))
        ths.append(th)
        time.sleep(np.random.rand())
        th.start()
    
    for th in ths :
        th.join()

### 예약자 리뷰 크롤링 함수 실행

In [0]:
review_list = []
rejected_salon_list = []
booking_crawler(start, end)
booking_reviewDF=DataFrame(review_list)
booking_reviewDF.to_csv("./naver_place_booking_review(%i-%i).csv" %(start, end), index=False)
reject_booking_salon.to_csv("./naver_reject_booking_salon(%i-%i).csv" %(start, end), index=False)

### 영수증 리뷰 크롤링 함수 실행

In [0]:
start=0
end=2727
n_thread = 10
review_list = []
rejected_salon_list = []
threading_craw(start, end, n_thread)
receipt_reviewDF=DataFrame(review_list)
receipt_reviewDF.to_csv("./receipt/naver_place_receipt_review(%i-%i).csv" %(start, end-1), sep=',', encoding='utf-8-sig')
reject_receipt_salon= DataFrame(rejected_salon_list)
reject_receipt_salon.to_csv("./receipt/naver_reject_receipt_salon(%i-%i).csv" %(start, end-1), sep=',', encoding='utf-8-sig')

  0%|                                                                                           | 0/10 [00:00<?, ?it/s]

0 준오헤어 노원사거리점 이미지가 없는 리뷰 저장.
0 준오헤어 노원사거리점 이미지가 없는 리뷰 저장.
0 준오헤어 노원사거리점 이미지가 없는 리뷰 저장.
0 준오헤어 노원사거리점 이미지가 없는 리뷰 저장.
0 준오헤어 노원사거리점 이미지가 없는 리뷰 저장.
0 준오헤어 노원사거리점 이미지가 없는 리뷰 저장.
0 준오헤어 노원사거리점 이미지가 없는 리뷰 저장.
0 준오헤어 노원사거리점 이미지가 없는 리뷰 저장.
0 준오헤어 노원사거리점 이미지가 없는 리뷰 저장.
0 준오헤어 노원사거리점 이미지가 없는 리뷰 저장.


100%|█████████████████████████████████████████████████████████████████████████████████| 10/10 [00:00<00:00, 526.18it/s]


0 준오헤어 노원사거리점 1/3 페이지 완료


  0%|                                                                                           | 0/10 [00:00<?, ?it/s]

...

887 리츠A 헤어살롱 1/0 페이지 완료
2264 케렌시아헤어 리뷰 데이터 없음
2265 오샤레 헤어살롱 리뷰 데이터 없음
1716 준헤어 리뷰 데이터 없음
1435 모드니살롱 리뷰 데이터 없음
2543 이영미 헤어 리뷰 데이터 없음
2266 피다헤어  미용실 리뷰 데이터 없음
1717 올드스캣 리뷰 데이터 없음
1718 살롱앤드 리뷰 데이터 없음
2267 에르아룸 리뷰 데이터 없음


  0%|                                                                                           | 0/10 [00:00<?, ?it/s]

2268 크랭크헤어스튜디오 청량리점 이미지가 없는 리뷰 저장.
2268 크랭크헤어스튜디오 청량리점 이미지가 없는 리뷰 저장.
2268 크랭크헤어스튜디오 청량리점 이미지가 없는 리뷰 저장.
2268 크랭크헤어스튜디오 청량리점 이미지가 없는 리뷰 저장.
2268 크랭크헤어스튜디오 청량리점 이미지가 없는 리뷰 저장.
2268 크랭크헤어스튜디오 청량리점 이미지가 없는 리뷰 저장.
2268 크랭크헤어스튜디오 청량리점 이미지가 없는 리뷰 저장.
2268 크랭크헤어스튜디오 청량리점 이미지가 없는 리뷰 저장.
2268 크랭크헤어스튜디오 청량리점 이미지가 없는 리뷰 저장.
2268 크랭크헤어스튜디오 청량리점 이미지가 없는 리뷰 저장.


100%|█████████████████████████████████████████████████████████████████████████████████| 10/10 [00:00<00:00, 666.49it/s]


2268 크랭크헤어스튜디오 청량리점 1/2 페이지 완료
51 진미장살롱 리뷰 데이터 불러오기 실패
